# Serving Alibi-Detect models

Out of the box, `mlserver` supports the deployment and serving of `alibi-detect` models.
By default, it will assume that these models have been [serialised using `joblib`](https://scikit-learn.org/stable/modules/model_persistence.html).

In this example, we will cover how we can train and serialise a simple model, to then serve it using `mlserver`.

## Training

The first step will be to train a simple `alibi-detect` model.
For that, we will use the [income Classifier example from the `alibi-detect` documentation](https://docs.seldon.io/projects/alibi-detect/en/latest/examples/cd_chi2ks_adult.html) which trains a drift detector.

In [1]:
import alibi
import matplotlib.pyplot as plt
import numpy as np

In [2]:
adult = alibi.datasets.fetch_adult()
X, y = adult.data, adult.target
feature_names = adult.feature_names
category_map = adult.category_map
X.shape, y.shape

((32561, 12), (32561,))

In [3]:
n_ref = 10000
n_test = 10000

X_ref, X_t0, X_t1 = X[:n_ref], X[n_ref:n_ref + n_test], X[n_ref + n_test:n_ref + 2 * n_test]
X_ref.shape, X_t0.shape, X_t1.shape

((10000, 12), (10000, 12), (10000, 12))

In [4]:
categories_per_feature = {f: None for f in list(category_map.keys())}
categories_per_feature

{1: None, 2: None, 3: None, 4: None, 5: None, 6: None, 7: None, 11: None}

### Saving our trained model

To save our trained model, we will serialise it using `joblib`.
While this is not a perfect approach, it's currently the recommended method to persist models to disk in the [`scikit-learn` documentation](https://scikit-learn.org/stable/modules/model_persistence.html).

Our model will be persisted as a file named `mnist-svm.joblib`

In [9]:
import pickle
filepath = 'alibi-detector-artifacts/ref_data.pkl'  # change to directory where detector is saved
pickle.dump(X_ref, open(filepath,"wb"))

## Serving

Now that we have trained and saved our model, the next step will be to serve it using `mlserver`. 
For that, we will need to create 2 configuration files: 

- `settings.json`: holds the configuration of our server (e.g. ports, log level, etc.).
- `model-settings.json`: holds the configuration of our model (e.g. input type, runtime to use, etc.).

### `settings.json`

In [6]:
%%writefile settings.json
{
    "debug": "true"
}

Overwriting settings.json


### `model-settings.json`

In [7]:
%%writefile model-settings.json
{
  "name": "income-classifier-cd",
  "implementation": "mlserver_alibi_detect.AlibiDetector",
  "parameters": {
    "uri": "./alibi-detector-artifacts/ref_data.pkl",
    "version": "v0.1.0"
  }
}

Overwriting model-settings.json


### Start serving our model

Now that we have our config in-place, we can start the server by running `mlserver start .`. This needs to either be ran from the same directory where our config files are or pointing to the folder where they are.

```shell
mlserver start .
```

Since this command will start the server and block the terminal, waiting for requests, this will need to be ran in the background on a separate terminal.

### Send test inference request

We now have our model being served by `mlserver`.
To make sure that everything is working as expected, let's send a request from our test set.

For that, we can use the Python types that `mlserver` provides out of box, or we can build our request manually.

In [8]:
from alibi_detect.cd import ChiSquareDrift, TabularDrift
cd = TabularDrift(X_ref, p_val=.05, categories_per_feature=categories_per_feature)
cd.predict(X_t0,drift_type="feature")

{'data': {'is_drift': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  'distance': array([1.1600000e-02, 8.4865131e+00, 4.7529316e+00, 3.1599441e+00,
         8.1941366e+00, 4.8458526e-01, 5.8652312e-01, 2.1689056e-01,
         2.4000001e-03, 1.6000000e-03, 1.1600000e-02, 9.9910326e+00],
        dtype=float32),
  'p_val': array([0.50786483, 0.38744345, 0.5758693 , 0.36761624, 0.41473988,
         0.99267685, 0.9645494 , 0.6414195 , 1.        , 1.        ,
         0.50786483, 0.44128036], dtype=float32),
  'threshold': 0.05},
 'meta': {'name': 'TabularDrift',
  'detector_type': 'offline',
  'data_type': None}}

In [10]:
import requests

x_0 = X_t0
inference_request = {
    "parameters": {"drift_type": "feature",},
    "inputs": [
        {
            "name": "predict",
            "shape": x_0.shape,
            "datatype": "FP32",
            "data": x_0.tolist(),
        }
    ],
}

endpoint = "http://localhost:8080/v2/models/income-classifier-cd/versions/v0.1.0/infer"
response = requests.post(endpoint, json=inference_request)

In [11]:
import json
response_dict = json.loads(response.text)
print(response_dict['parameters'],"\n")

labels = ['No!', 'Yes!']
for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = feature_names[f]
    is_drift = response_dict['outputs'][0]['data'][f]
    # stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]}')

{'content_type': None, 'name': 'TabularDrift', 'data_type': None, 'detector_type': 'offline'} 

Age -- Drift? No!
Workclass -- Drift? No!
Education -- Drift? No!
Marital Status -- Drift? No!
Occupation -- Drift? No!
Relationship -- Drift? No!
Race -- Drift? No!
Sex -- Drift? No!
Capital Gain -- Drift? No!
Capital Loss -- Drift? No!
Hours per week -- Drift? No!
Country -- Drift? No!
